In [ ]:
# Lucas Xu, Highschool Intern @ UAlberta
#Python 3.11.9, WindowsOS 11 
#UI of a heatmap designed to visualize the amount of people per room
#main var/classes: class PannableCanvas, root, button_zoomin, button_zoomout, slider
import tkinter as tk
import tkinter.font as tkFont
from PIL import Image, ImageTk, ImageDraw
from tkinter import filedialog
from tkinter import simpledialog
from tkinter import messagebox
from tkinter import ttk
import fitz
import csv
import json
import pandas as pd
import os
import tempfile
import shutil

# window init
root = tk.Tk()
root.title("HeatMap UI")
root.geometry("1600x1000")
root.config(bg='white', pady=100, padx=100)
root.state('zoomed')

max_width = 1600
max_height = 800

#init filename database
filename_database = set()
filename_dict = {}

#init root directory
os.chdir('/')

#temp dir init
temp_dir = tempfile.mkdtemp()

#room data dir init
#Have the path where the executable file will be when further implemented
roomdata_path = r'Users\plast\Documents\JupyterLab\roomdata'
os.makedirs(roomdata_path, exist_ok=True)


#Canvas init, canvas to hold a pannable image of the map
class PannableCanvas(tk.Frame):
    #define parameters and initialize
    def __init__(self, master, image_path, max_width, max_height, current_hour, data_path, *args, **kwargs):

        #init parent class holding canvas
        super().__init__(master,*args,**kwargs)

        #create class args to be made public
        self.image_path = image_path
        self.data_path = data_path
        self.max_width = max_width
        self.max_height = max_height
        self.current_scale = 1.0
        self.tk_image = None
        self.roomdata_path = None
        self.rooms = None
        self.data = None
        self.savestate = {}
        self.memorystates = []
        self.current_hour = current_hour
        self.display_height = max_height
        self.display_width = max_width
        self.shapes = []
        self.vertices = []
        self.lines = []
        self.mouseline = None
        self.is_painting = False
        self.is_erasing = False
        self.orientation = 1
        
        #canvas init
        self.canvas = tk.Canvas(self, width=max_width, height=max_height, bg='white')
        self.canvas.grid(row=0,column=0, sticky = "nsew")
        
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        #image init
        self.image = self.scale_image(image_path, max_width, max_height, self.current_scale)

        self.canvas_image = self.canvas.create_image(0,0, anchor=tk.NW, image=self.image)
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))
        
        
        #mouse init
        self.key_rebind()

        #center canvas
        if self.image != None:
            self.center_canvas()

    #functions to determine x,y coordinates based off the current orientation
    def rotate_0(self, x, y):
        return x, y

    def rotate_90(self, x, y):
        return y, -x

    def rotate_180(self, x, y):
        return -x, -y

    def rotate_270(self, x, y):
        return -y, x
    
    def translate_coordinates(self, x, y):
        if self.orientation == 1:
            return self.rotate_0(x, y)
        elif self.orientation == 2:
            return self.rotate_90(x, y)
        elif self.orientation == 3:
            return self.rotate_180(x, y)
        elif self.orientation == 4:
            return self.rotate_270(x, y)
    
    #a function wrapper to run 2 functions at once for event binding
    def draw_wrapper(self, event):
        self.hover_over(event)
        self.draw_mouseline(event)

    #using the roomdata path, find current existing csv or create csv of rooms of that floor
    def extract_rooms(self):
        try:
            df = pd.read_csv(self.roomdata_path)
            df['coords'] = df['coords'].apply(json.loads)
        except:
            df = pd.DataFrame(columns = ['id', 'coords'])
        if list(df.columns) != ['id', 'coords']:
            result = tk.messagebox.askyesno(title="Invalid Format", message="Invalid csv columns. Do you want to delete and replace all of the current floor's room data?")
            if result:
                df = pd.DataFrame(columns = ['id', 'coords'])
            else:
                self.image_path = ''
                self.update_image()
                df = None
                return
        
        self.savestate = df.to_dict(orient = 'list')
        self.rooms = df
        self.update_image()

    #extract the data of how many people are in a room
    def extract_data(self):
        try:
            df = pd.read_csv(self.data_path)
        except:
            tk.messagebox.showwarning(title="File not Found", message="Invalid path to file, file not found.")
            return

        #'people' column will be a list, with each index representing an hour 0 = 0:00,  23 = 23:00
        if list(df.columns) != ['id', 'people']:
            tk.messagebox.showwarning(title="Invalid Format", message="FIle format is invalid, file not used.")
            return

        self.update_image()
    
    #rebind keys when paint mode is on/off
    def key_rebind(self):
        #include shift for later when grid snapping is worked on
        if self.is_painting:
            #code for paint mode
            self.canvas.unbind("<ButtonPress-1>")
            self.canvas.unbind("<B1-Motion>")
            self.canvas.unbind('<ButtonPress-3>')
            self.canvas.unbind('<KeyRelease>')
            self.canvas.unbind('<Control-z>')
            self.canvas.unbind('<Control-y>')
            self.canvas.bind("<ButtonPress-1>", self.start_pan)
            self.canvas.bind("<B1-Motion>", self.pan)
            self.canvas.bind("<ButtonPress-3>", self.create_vertex)
            self.canvas.bind("<Motion>", self.draw_wrapper)
            self.bind("<KeyRelease>", self.can_create)
            # self.canvas.bind("<Control-z>", self.undo)
            # self.canvas.bind("<Control-y>", self.redo)
        elif self.is_erasing:
            self.canvas.unbind("<ButtonPress-1>")
            self.canvas.unbind("<B1-Motion>")
            self.canvas.unbind('<ButtonPress-3>')
            self.canvas.unbind('<KeyRelease>')
            self.canvas.unbind('<Control-z>')
            self.canvas.unbind('<Control-y>')
            self.canvas.unbind("<Motion>")
            self.canvas.bind("<Motion>", self.hover_over)
            self.canvas.bind("<ButtonPress-3>", self.delete_shape)
            self.canvas.bind("<ButtonPress-1>", self.start_pan)
            self.canvas.bind("<B1-Motion>", self.pan)
        else:
            #code for not paint mode
            self.canvas.unbind("<ButtonPress-1>")
            self.canvas.unbind("<B1-Motion>")
            self.canvas.unbind('<ButtonPress-3>')
            self.canvas.unbind('<KeyRelease>')
            self.canvas.unbind('<Control-z>')
            self.canvas.unbind('<Control-y>')
            self.canvas.unbind("<Motion>")
            self.canvas.bind("<Motion>", self.hover_over)
            self.canvas.bind("<ButtonPress-1>", self.start_pan)
            self.canvas.bind("<B1-Motion>", self.pan)
            
      #scale the image to fit the canvas
    #HEIGHT AND WIDTH SWITCHED DUE TO THE 90 DEGREE ROTATE      
    def scale_image(self, image_path, max_width, max_height, scale):
        image_path = image_path.strip().strip("'").strip('"')
        try:
            image = Image.open(image_path)
        except Exception as e:
            print(f"Error opening image: {e}")
            return None
            
        original_width, original_height = image.size

        #recalculate width and heigh via aspect ratio
        aspect_ratio = original_width/original_height

        dw = max_width - original_width
        dh = max_height - original_height
        
        if dw > dh:
            new_width = max_height
            new_height = new_width / aspect_ratio
        else:
            new_height = max_width
            new_width = new_height * aspect_ratio

        new_height = int(new_height * scale)
        new_width = int(new_width * scale)

        self.display_height = new_width
        self.display_width = new_height
        
        #resize using calculated dimensions
        #a lot of the images are vertically oriented, so 90 degree rotation is applied as a standard
        new_image = image.resize((new_width, new_height), Image.LANCZOS)
        new_image = new_image.rotate(-90 * self.orientation, expand=True)
        tk_image = ImageTk.PhotoImage(new_image)
        return tk_image

    #redraw image and all shapes for when zoomin/zoomout is called or hour change
    def update_image(self):
        self.image = self.scale_image(self.image_path, self.max_width, self.max_height, self.current_scale)
        if self.image:
            self.canvas.itemconfig(self.canvas_image, image=self.image)
            self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))
        else:
            print("No image loaded")
        
        #delete old shapes
        for shape in self.shapes:
            self.canvas.delete(shape['id'])

        self.shapes.clear()

        #create new shapes
        #implement colour later
        if not self.rooms.empty:
            for _, row in self.rooms.iterrows():
                self.draw_shapes(row)

        self.draw_vertices()

    #zoom in
    def zoom_in(self):
        if self.current_scale * 1.2 <= 3:
            self.current_scale *= 1.2 
            self.update_image()
    #zoom out
    def zoom_out(self):
        if self.current_scale / 1.2 >= 1.0:
            self.current_scale /= 1.2
            self.update_image()
            
    #track starting coords
    def start_pan(self, event):
        self.canvas.scan_mark(event.x, event.y)
        
    #drag with scan_dragto
    def pan(self, event):
        self.canvas.scan_dragto(event.x, event.y, gain=1)
        
    def create_vertex(self, event):
        #mark down canvas coordinates on where clicks are and append to self.vertices
        canvas_x = self.canvas.canvasx(event.x)
        canvas_y = self.canvas.canvasy(event.y)
        #calculate the original coordinates if the image is scaled or not
        canvas_x = canvas_x / self.current_scale
        canvas_y = canvas_y / self.current_scale
        self.vertices.append(canvas_x)
        self.vertices.append(canvas_y)

        self.update_image()


    def draw_vertices(self):
        #clear all lines so that lines do not get drawn twice
        for line in self.lines:
            self.canvas.delete(line)

        #scale the new coordiantes based off the scale
        scaled_coords = [coord * self.current_scale for coord in self.vertices]
        
        #connect vertices from one to another in the list, from the bottom to top
        for i in range(int(len(self.vertices) / 2.0) - 1):
            cursor = i*2
            if len(self.vertices) > 1:
                line = self.canvas.create_line(scaled_coords[cursor], scaled_coords[cursor+1], scaled_coords[cursor+2], scaled_coords[cursor+3], fill='black', width=2)
                self.lines.append(line)

        #grid snapping can be worked on later

    #while actively drawing a box, connect the last drawn vertex to mouse
    def draw_mouseline(self, event):
        if not self.vertices:
            return
            
        startx = self.vertices[-2] * self.current_scale
        starty = self.vertices[-1] * self.current_scale
        endx = self.canvas.canvasx(event.x)
        endy = self.canvas.canvasy(event.y) 

        #clear previous line
        self.canvas.delete(self.mouseline)

        #draw new line
        self.mouseline = self.canvas.create_line(startx, starty, endx, endy, fill="black", width=1)

    #raycast algorithm to detect whether mouse is in a shape
    def raycast_detect(self, x, y):
        for room in self.savestate['coords']:
            
            scaled_room = [coord * self.current_scale for coord in room]

            
            
            n = int(len(scaled_room) / 2)
            inside = False

            #first pair of vertex coords
            px, py = scaled_room[0], scaled_room[1]

            for i in range(1, n+1):
                #next pair of vector coords, if last vector, it becomes the first vector to connect the last pair of coords with the start
                qx, qy = scaled_room[2 * (i % n)], scaled_room[2 * (i % n) + 1]

                #check is x,y coords are wthin y values and on the left of the current side of the polygon
                if y > min(py, qy):
                    if y < max(py, qy):
                        if x <= max(px, qx):
                            if py != qy:
                                xinters = (y - py) * (qx - px) / (qy - py) + px
                            if px == qx or x <= xinters:
                                inside = not inside

                px,py = qx,qy

            if inside:
                #return the INDEX of the room in self.savestate
                return self.savestate['coords'].index(room)

        return None
                
            
    def delete_shape(self, event):
        canvas_x = self.canvas.canvasx(event.x)
        canvas_y = self.canvas.canvasy(event.y)

        active_room = self.raycast_detect(canvas_x, canvas_y)
        #delete elements of selected room
        if active_room != None:
            del self.savestate['id'][active_room]
            del self.savestate['coords'][active_room]
            self.rooms.drop(active_room, inplace=True)
            self.rooms = self.rooms.reset_index(drop=True)
            self.update_image()
    
    #detect whether the mouse is hovered over a shape using ray casting, with hover, display room id and # of ppl, in delete mode, right clicking will delete
    def hover_over(self, event):
        if self.image:
            global label
            canvas_x = self.canvas.canvasx(event.x)
            canvas_y = self.canvas.canvasy(event.y)

            active_room = self.raycast_detect(canvas_x, canvas_y)

            #Add number of people later
            if active_room != None:
                txt_roomid = self.savestate['id'][active_room]
                txt_roomppl = None

                label.pack_forget()
                label = tk.Label(top_left, text=f"Room Id: {txt_roomid} # of People: {txt_roomppl}", width= 40, height=4, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', font=("Arial", 12))
                label.pack(side='left', padx=50, pady=5)
            else:
                label.pack_forget()
                label = tk.Label(top_left, text="Room Id:  # of People: ", width= 40, height=4, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', font=("Arial", 12))
                label.pack(side='left', padx=50, pady=5)

    
    #check if shape can be created based off key-release
    def can_create(self, event):
        if event.keysym == 'Return':    
            if len(self.vertices) > 4:
                self.create_shape()
                
    #take current placed vertices and create a shape that can be pushed into csv and pandas df for other usages
    def create_shape(self):
        #take a room id to assign the shape being made
        room_id = None
        while not room_id:
            room_id = simpledialog.askstring("Input", "Enter room id:")
            if room_id is None:  # The user canceled the input
                messagebox.showwarning("Warning", "Operation cancelled")
                return
            elif not room_id.strip():  # The user entered an empty string
                messagebox.showwarning("Warning", "Input cannot be blank!")
                room_id = None
            elif room_id in self.savestate['id']:
                messagebox.showwarning("Warning", "Already existing input, please enter a different id")
                room_id = None
        
        #savestate is the current loaded memory state, update savestate and push into a pandas dataframe to be later saved
        self.savestate['id'].append(room_id)
        self.savestate['coords'].append(self.vertices)
        self.rooms = pd.DataFrame(self.savestate)
        
        #clear vertices + update
        for line in self.lines:
            self.canvas.delete(line)
        self.vertices = []
        self.canvas.delete(self.mouseline)
        self.update_image()

        #calculate colour of shape from green to red based off of how many people
    def calc_color(self, id):
        #scale of 1-10, 10 being max power
        if self.rooms.empty or self.rooms is None:
            room = self.data[self.data['id'] == id]
        else:
            return None
        try:
            n = room['people'][self.current_hour]
        except:
            messagebox.showwarning("Invalid Format", "Data could not be extracted properly, invalid data or invalid format")
            return None
            
        val = min(n/10, 1)
        red = int(val * 255)
        green = int((1-val) * 255)

        #translate into hex
        color = f'#{red:02x}{green:02x}00'
        return color

    
    #draw all rooms of the current hour to be overlayed on the image using coordinates and draw.polygon()
    #going to need another parameter, "people" to take in a pandas object to associate # of ppl per room
    def draw_shapes(self, room):
        
        # #init values for polygon
        color = self.calc_color(room['id'])
        alpha = 128
        coords = [coord * self.current_scale for coord in room['coords']]
        
        #create image with RGBA values for translucent effect
        img = Image.new('RGBA', (self.display_width, self.display_height), (0, 0, 0, 0))
        draw = ImageDraw.Draw(img)

        #fill the image with a polygon of calculated colour
        #polygons by default do not take alpha values, hence the image was created
        if color != None:
            draw.polygon(coords, fill=(int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16), alpha), outline='black')
        else:
            draw.polygon(coords, fill=(255,0,0,128), outline='black')
            
        tk_img = ImageTk.PhotoImage(img)

        image_item = self.canvas.create_image(0, 0, anchor=tk.NW, image=tk_img)

        #add shapes into list for easy access when updating
        self.shapes.append({'id': image_item, 'original_coords': coords, 'image': tk_img})

    #when finished with paint mode, save the pandas dataframe into a csv file in roomdata_path
    def save_file(self):
        self.rooms.to_csv(self.roomdata_path, index=False)
    
    #center the canvas
    def center_canvas(self):
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        image_width = self.image.width()
        image_height = self.image.height()

        if image_width > canvas_width:
            self.canvas.xview_moveto((image_width - canvas_width) / 2 / image_width)
        else:
            self.canvas.xview_moveto(0.5 - canvas_width / image_width / 2)

        if image_height > canvas_height:
            self.canvas.yview_moveto((image_height - canvas_height) / 2 / image_height)
        else:
            self.canvas.yview_moveto(0.5 - canvas_height / image_height / 2)

        # Adjust the scroll region to accommodate zoom
        self.canvas.config(scrollregion=self.canvas.bbox(tk.ALL))

#create expandable section in the listbox
class ExpandableSection():
    def __init__(self, master, title, content, *args, **kwargs):
        super().__init__(master, *args, **kwargs)
        #self args init
        self.title = title
        self.content = content
        self.collapsed = True

        #create button for each creation of class when called
        if self.title != None:
            self.title_button = tk.Button(self, text=title, height = 2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=self.toggle)
            self.title_button.pack(fill='x')

        #create frame for content
        self.content_frame = tk.Frame(self)
            
        self.create_content()

    #fill new listbox value with content, recursive function for any sub directories
    def create_content(self):
        if isinstance(self.content, dict):
        # Content is a dictionary of sub-sections
            for sub_title, sub_content in self.content.items():
                sub_section = ExpandableSection(self.content_frame, sub_title, sub_content)
                sub_section.pack(fill='x', pady=5, padx=20)
        elif isinstance(self.content, list):
            # Content is a list of options
            for option in self.content:
                button = tk.Button(self.content_frame, text=option)
                button.pack(fill='x')
        elif isinstance(self.content, str):
            button = tk.Button(self.content_frame, text=option)

    def on_button_click(self, option):
        parent_titles = []
        parent = self
        while parent is not None:
            parent_titles.append(parent.title)
            parent = parent.parent
        parent_titles.reverse()
        parent_path = os.path.sep.join(parent_titles)
        print(f"Clicked on {option}. Full path: {parent_path}/{option}")

    
    #toggle collapsed on and off    
    def toggle(self):
        if self.collapsed:
            self.content_frame.pack(fill='x')
        else:
            self.content_frame.forget()
            self.collapsed = not self.collapsed            

#Init function to create the scrollable listbox of buttons
def create_scrollbox(container):
    canvas = tk.Canvas(container)
    scrollbar = tk.Scrollbar(container, orient="vertical", command=canvas.yview)
    scrollable_frame = tk.Frame(canvas)
    
    scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    # scrollable_frame.bind("<>", ) later

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    return scrollable_frame    

def zoom_out():
    canvas.zoom_out()

def zoom_in():
    canvas.zoom_in()

#update value of the slider to show the data of specified hour
def slider_change(value):
    canvas.current_hour = int(value)
    canvas.update_image()

#function for changing the hour for button_change()
def change_hour():
    global after_id
    current_val = slider.get()
    if current_val == 23:
        current_val = 0
    if button_video.image == pause_img:
        if current_val + 1 < 24:
            current_val += 1
            slider.set(current_val)
            canvas.update_image()
            after_id = root.after(1000, change_hour)
        else:
            return
  
#play pause function for play pause button
def button_change():
    global after_id
    #if the images are currently paused
    if button_video.image == play_img:
        button_video.config(image = pause_img)
        button_video.image = pause_img
        change_hour()
    else:
        #if the iamges are currently playing
        button_video.config(image = play_img)
        button_video.image = play_img
        if after_id is not None:
            root.after_cancel(after_id)  # Cancel the scheduled increment
            after_id = None

#rotating the image for the rotate button
def button_rotate():
    #delete later
    return
    if canvas.image:
        canvas.orientation += 1
        if canvas.orientation > 4:
            canvas.orientation = 1
        canvas.update_image()

#create a dict to send to the scrollbox
def build_directory_dict(root_dir):
    directory_dict = {}

    for root, dirs, files in os.walk(root_dir):
        # Create relative path for this directory
        relative_root = os.path.relpath(root, root_dir)
        
        # Get the dictionary for this directory
        current_dict = directory_dict
        if relative_root != '.':
            parts = relative_root.split(os.sep)
            for part in parts:
                if part not in current_dict:
                    current_dict[part] = []
                current_dict = current_dict[part]

        # Add files directly under the current directory
        for file in files:
            if file.lower().endswith('.png'):
                if not isinstance(current_dict, list):
                    current_dict = current_dict if isinstance(current_dict, dict) else {}
                if isinstance(current_dict, dict):
                    current_dict.setdefault('files', []).append(file)
            elif file.lower().endswith('.pdf'):
                # Handle PDF files: convert pages to PNGs and store them
                pdf_path = os.path.join(root, file)
                pdf = fitz.open(pdf_path)
                for i in range(len(pdf)):
                    img = pdf.load_page(i).get_pixmap()
                    newpdf_name = f'page{i+1}_{file}.png'
                    newpdf_path = os.path.join(temp_dir, newpdf_name)
                    img.save(newpdf_path)
                    if not isinstance(current_dict, list):
                        current_dict = current_dict if isinstance(current_dict, dict) else {}
                    if isinstance(current_dict, dict):
                        current_dict.setdefault('files', []).append(newpdf_name)
                pdf.close()

    return directory_dict
        

def update_filename_dict(d, root, base_path=''):
    for key, value in d.items():
        current_pathname = os.path.join(base_path, key)
        if isinstance(value, dict):
            update_filename_dict(value, current_pathname)
        else:
            filename_dict[current_pathname] = os.path.join(root, key)

#button function to upload directoy
def upload_dir():
    dirname_data = filedialog.askdirectory(title="Select a directory")
    if not dirname_data:
        return
    
    # Create the directory structure dictionary
    directory_dict = build_directory_dict(dirname_data)

    update_filename_dict(directory_dict, dirname_data)

    print(directory_dict)
    
    for dir_title, content in directory_dict.items():
        section = ExpandableSection(scrollbox, dir_title, content)
                    
    
#requests a file destination input from user, stores path into database
def upload_image():
    filename_data = filedialog.askopenfilename(title="Select a file", filetypes=(("PNG", "*.png"), ("PDF", "*.pdf")))
    if not filename_data:
        return
    
    #check if pdf
    name = os.path.basename(filename_data)
    #this will have to change as the user changes
    _, extension = os.path.splitext(filename_data)
    
    if extension == ".pdf":
        #convert from pdf into jpg
        pdf = fitz.open(filename_data)
        for i in range(len(pdf)): 
            #save output of jpg
            page = pdf.load_page(i)
            img = page.get_pixmap()
            newpdf_name = f'page{i+1}_{name}.png'
            newpdf_path = os.path.join(temp_dir, newpdf_name)
            img.save(newpdf_path)
            section = ExpandableSection(scrollbox, None, newpdf_name)
            section.pack(fill='x')
            if newpdf_name not in filename_dict:
                filename_dict[newpdf_name] = newpdf_path
    else:
        filename_database.add(name)
        if name not in filename_dict:
           filename_dict[name] = filename_data 

def upload_data():
    filename_data = filedialog.askopenfilename(title="Select a file", filetypes=(("CSV", "*.csv"),))
    canvas.data_path = filename_data
    canvas.extract_data()

#grab selection in combobox, and then display on canvas
def display_selected(event):
    global combobox
    global canvas
    #grab the filename from combobox
    option_name = combobox.get()
    #using filename as a key, get filename's path
    option = filename_dict[option_name]
    #set canvas roomdata path to represent the combobox selection
    canvas.roomdata_path = os.path.join(roomdata_path, os.path.splitext(option_name)[0]+'.csv')
    canvas.extract_rooms()
    canvas.image_path = option
    canvas.update_image()
    canvas.center_canvas()
    canvas.pack(fill=tk.BOTH, expand=True)
    
#enable painting mode and its features
def paint_mode():
    global paint_button
    global erase_button
    global save_button
    
    if combobox.get() == '':
        tk.messagebox.showwarning(title="Missing Image", message="Missing image, please upload an image first.")
    else:
        #If paint mode is ON:
        if canvas.is_painting or canvas.is_erasing:
            canvas.is_painting = False
            canvas.is_erasing = False

            #button pack and unpack
            erase_button.pack_forget()
            save_button.pack_forget()
            paint_button = tk.Button(right_frame, text = "Paint", width= 10, height = 2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=paint_mode)
            paint_button.pack(side = 'bottom', pady = 10)
        else:
            #if paint mode is OFF:
            canvas.is_painting = True
            canvas.focus_set()

            #button pack and unpack
            paint_button.pack_forget()
            #code erase mode for erase_button later
            save_button = tk.Button(right_frame, text= "Save & Exit", width=10, height=2, relief="solid", bg="white", activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', command=save_exit)
            save_button.pack(padx=5, pady=5)
            erase_button = tk.Button(right_frame, text = "Erase", width=10, height = 2, relief="solid", bg="white", activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', command=erase_mode)
            erase_button.pack(padx=5, pady=5)

    canvas.key_rebind()

#enable erase mode
def erase_mode():
    global erase_button
    if canvas.is_erasing:
        erase_button.pack_forget()
        erase_button = tk.Button(right_frame, text = "Erase", width= 10, height = 2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=erase_mode)
        erase_button.pack(padx=5, pady = 5)
        canvas.is_erasing = False
        canvas.is_painting = True
    else:
        erase_button.pack_forget()
        erase_button = tk.Button(right_frame, text = "Paint", width=10, height = 2, relief="solid", bg="white", activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', command=erase_mode)
        erase_button.pack(padx=5, pady=5)
        canvas.is_erasing = True
        canvas.is_painting = False

    canvas.key_rebind()

def save_exit():
    paint_mode()
    canvas.save_file()

#update the combobox values
def update_combobox():
    combobox['values'] = list(filename_database)

def defocus(event):
    event.widget.master.focus_set()

#window close protocols
def on_close():
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        root.destroy()

# Load and display an image
image_path = ""
data_path = ""

#frame and button init
top_frame = tk.Frame(root, bg = 'white')
top_frame.pack(side = tk.TOP)
top_left = tk.Frame(top_frame, bg='white')
top_left.pack(side = "left")

rotate_path = Image.open(r'C:\Users\plast\Documents\JupyterLab\rotate-left.png')
rotate_path = rotate_path.resize((30,30), Image.LANCZOS)
rotate_img = ImageTk.PhotoImage(rotate_path)
button_rotate = tk.Button(top_frame, image=rotate_img,  width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_rotate)
button_rotate.pack(side = 'right', padx=10, pady=10)

button_zoomin = tk.Button(top_frame, text = "+", width=4, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=zoom_in)
button_zoomin.pack(side = 'right', expand = False, pady=10)
button_zoomout = tk.Button(top_frame, text = "-", width=4, height=2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=zoom_out)
button_zoomout.pack(side = 'right', expand = False, pady=10)

right_frame = tk.Frame(root, bg = 'white')
right_frame.pack(side = tk.RIGHT, padx = 10)
right_bottom = tk.Frame(right_frame, bg = 'white')
right_bottom.pack(side="bottom")

#combobox for map selection
# combobox = ttk.Combobox(right_frame)
# combobox.bind("<<ComboboxSelected>>", display_selected)
# combobox.pack(side = 'top', padx=10)

#scrollable listbox 
scrollbox = create_scrollbox(right_frame)


#Paint mode button
paint_button = tk.Button(right_bottom, text = "Paint", width= 10, height = 2, relief='solid',bg='white', activebackground='white', activeforeground='black',  borderwidth=1, highlightbackground = 'grey', command=paint_mode)
paint_button.pack(side = 'bottom', pady = 10)

#label for captions 
label = tk.Label(top_left, text="Room Id:  # of People: ", width= 40, height=4, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground='grey', font=("Arial", 12))
label.pack(side='left', padx=10, pady=5)

#load images for play/pause button
play_path = Image.open(r'C:\Users\plast\Documents\JupyterLab\play.png')
play_path = play_path.resize((35,35), Image.LANCZOS)
pause_path = Image.open(r'C:\Users\plast\Documents\JupyterLab\pause.png')
pause_path = pause_path.resize((35,35), Image.LANCZOS)
play_img = ImageTk.PhotoImage(play_path)
pause_img = ImageTk.PhotoImage(pause_path)
button_video = tk.Button(top_frame, image=play_img, width=35, height=35, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=button_change)
button_video.image = play_img
button_video.pack(side = 'left', pady=10)

#upload dir with images
uploaddir_button = tk.Button(top_frame, text="Upload Directory", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_dir)
uploaddir_button.pack(side = "left", pady=10, padx =10)

#Upload image button
uploadimg_button = tk.Button(top_frame, text="Upload Image", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_image)
uploadimg_button.pack(side = "left", pady=10, padx =10)

#upload room ppl data
uploaddata_button = tk.Button(top_frame, text="Upload Data", width=13, height=2, relief='solid', bg='white', activebackground='white', activeforeground='black', borderwidth=1, highlightbackground = 'grey', command=upload_data)
uploaddata_button.pack(side= 'left', pady=10)

slider = tk.Scale(top_frame, from_ = 0, to=23, orient='horizontal', command=slider_change, length = 300)
slider.pack(pady=20, padx = 20)



hour = 0

# #canvas init
canvas = PannableCanvas(root, image_path, max_width, max_height, hour, data_path)
canvas.pack(fill=tk.BOTH, expand=True)

slider.config(resolution= 1)

#window close protocols
root.protocol("WM_DELETE_WINDOW", on_close)

# Run the main event loop
root.mainloop()

Error opening image: [Errno 2] No such file or directory: 'C:\\'
{'Administration (51030)': [], 'Assiniboia Hall (51400)': [], 'Athabasca Hall (51450)': [], 'CCIS (53480)': [], 'CCIS-II - ESB Link (53483)': [], 'CCIS2 (53482)': [], 'CCIS2-NLT (53485)': [], 'Central Academic Building(CAB)(51850)': [], 'Chemistry (51990 + 52000)': [], 'Pembina Hall (53350)': [], 'South Academic Building(SAB)(52250)': [], 'Students Union Building(SUB)(54400)': [], 'Universiade Pavilion (54320)': []}


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\plast\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\plast\AppData\Local\Temp\ipykernel_7620\584097054.py", line 696, in upload_dir
    section = ExpandableSection(scrollbox, dir_title, content)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\plast\AppData\Local\Temp\ipykernel_7620\584097054.py", line 513, in __init__
    super().__init__(master, *args, **kwargs)
TypeError: object.__init__() takes exactly one argument (the instance to initialize)


In [1]:
import tkinter as tk
from tkinter import ttk

class ExpandableSection(tk.Frame):
    def __init__(self, master, title, content, parent=None, *args, **kwargs):
        super().__init__(master, *args, **kwargs)
        self.title = title
        self.content = content
        self.collapsed = True
        self.parent = parent  # Reference to the parent section

        self.title_button = ttk.Button(self, text=title, command=self.toggle)
        self.title_button.pack(fill='x')

        self.content_frame = tk.Frame(self)
        
        self.create_content()

    def create_content(self):
        if isinstance(self.content, dict):
            # Content is a dictionary of sub-sections
            for sub_title, sub_content in self.content.items():
                sub_section = ExpandableSection(self.content_frame, sub_title, sub_content, parent=self)
                sub_section.pack(fill='x', pady=5, padx=20)
        elif isinstance(self.content, list):
            # Content is a list of options
            for option in self.content:
                button = ttk.Button(self.content_frame, text=option, command=lambda opt=option: self.on_button_click(opt))
                button.pack(fill='x')

    def on_button_click(self, option):
        # Called when an option button is clicked
        print(f'Button "{option}" clicked')
        parent_titles = []
        parent = self
        while parent is not None:
            parent_titles.append(parent.title)
            parent = parent.parent
        parent_titles.reverse()
        parent_path = '/'.join(parent_titles)
        print(f'Parent path: {parent_path}')

    def toggle(self):
        if self.collapsed:
            self.content_frame.pack(fill='x')
        else:
            self.content_frame.forget()
        self.collapsed = not self.collapsed

def create_scrollable_frame(container):
    canvas = tk.Canvas(container)
    scrollbar = ttk.Scrollbar(container, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    return scrollable_frame

root = tk.Tk()
root.title("Expandable Sections Example")

container = ttk.Frame(root)
container.pack(fill="both", expand=True)

scrollable_frame = create_scrollable_frame(container)

# Example of a multi-dimensional dictionary
sections = {
    "Fruits": {
        "Citrus": ["Orange", "Lemon", "Lime"],
        "Berries": ["Strawberry", "Blueberry", "Raspberry"],
    },
    "Vegetables": {
        "Root": {
            "Carrot Family": ["Carrot", "Parsnip"],
            "Beet Family": ["Beetroot", "Sugar beet"],
        },
        "Leafy": ["Spinach", "Lettuce", "Kale"],
    },
    "Dairy": ["Milk", "Cheese", "Yogurt"],
}

for section_title, content in sections.items():
    section = ExpandableSection(scrollable_frame, section_title, content)
    section.pack(fill="x", pady=5, padx=5)

root.mainloop()


In [2]:
import os
from tkinter import filedialog
import fitz  # PyMuPDF

def build_directory_dict(root_dir):
    directory_dict = {}
    
    for root, dirs, files in os.walk(root_dir):
        # Create relative path for this directory
        relative_root = os.path.relpath(root, root_dir)
        
        # Get the dictionary for this directory
        current_dict = directory_dict
        if relative_root != '.':
            parts = relative_root.split(os.sep)
            for part in parts:
                if part not in current_dict:
                    current_dict[part] = {}
                current_dict = current_dict[part]
        
        # Add files to the dictionary
        for file in files:
            if file.lower().endswith('.png'):
                current_dict[file] = os.path.relpath(os.path.join(root, file), root_dir)
            elif file.lower().endswith('.pdf'):
                # Handle PDF files: convert pages to PNGs and store them
                pdf_path = os.path.join(root, file)
                pdf = fitz.open(pdf_path)
                for i in range(len(pdf)):
                    img = pdf.load_page(i).get_pixmap()
                    newpdf_name = f'page{i+1}_{file}.png'
                    newpdf_path = os.path.join(temp_dir, newpdf_name)
                    img.save(newpdf_path)
                    current_dict[newpdf_name] = os.path.relpath(newpdf_path, root_dir)
                pdf.close()
    
    return directory_dict

def print_dict(d, base_path=''):
    for key, value in d.items():
        current_path = os.path.join(base_path, key)
        if isinstance(value, dict):
            print_dict(value, current_path)
        else:
            # Print the relative path for files
            print(current_path)

def upload_dir():
    dirname_data = filedialog.askdirectory(title="Select a directory")
    if not dirname_data:
        return

    dirname_data = dirname_data.replace('/', '\\')
    
    print(dirname_data)
    
    # Create the directory structure dictionary
    directory_dict = build_directory_dict(dirname_data)
    
    # Print the multi-dimensional dictionary with paths joined by '\'
    print_dict(directory_dict)

# Ensure temp_dir exists or create it as needed
temp_dir = "temp"
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

# Call the upload_dir function to select a directory and build the dictionary
upload_dir()


C:\Users\plast\Downloads\wifiscans-main\wifiscans-main\floormaps
Administration (51030)\1st Floor.png
Administration (51030)\2nd Floor.png
Administration (51030)\3rd Floor.png
Administration (51030)\4th Floor.png
Administration (51030)\B1.png
Assiniboia Hall (51400)\1st Floor.png
Assiniboia Hall (51400)\2nd Floor.png
Assiniboia Hall (51400)\3rd Floor.png
Assiniboia Hall (51400)\4th Floor.png
Assiniboia Hall (51400)\Sub Basement.png
Athabasca Hall (51450)\1st Floor.png
Athabasca Hall (51450)\2nd Floor.png
Athabasca Hall (51450)\3rd Floor.png
Athabasca Hall (51450)\4th Floor.png
CCIS (53480)\B1.png
CCIS (53480)\B2.png
CCIS-II - ESB Link (53483)\CCIS-II - ESB Pedway.png
CCIS2 (53482)\1st Floor.png
CCIS2 (53482)\2nd Floor.png
CCIS2 (53482)\3rd Floor.png
CCIS2 (53482)\4th Floor.png
CCIS2 (53482)\5th Floor.png
CCIS2 (53482)\6th Floor.png
CCIS2 (53482)\7th Floor.png
CCIS2 (53482)\L1.png
CCIS2 (53482)\L2.png
CCIS2-NLT (53485)\1st Floor.png
CCIS2-NLT (53485)\L1.png
CCIS2-NLT (53485)\L2.png
Cent